In [21]:
import os 
# import openai 
from openai import OpenAI

In [3]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [4]:
import pandas as pd 

In [5]:
df = pd.read_csv("sales_data_sample.csv")

In [6]:
df.head()

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,...,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE
0,10107,30,95.70,2,2871.00,2/24/2003 0:00,Shipped,1,2,2003,...,897 Long Airport Avenue,NaN,NYC,NY,10022,USA,NaN,Yu,Kwai,Small
1,10121,34,81.35,5,2765.90,5/7/2003 0:00,Shipped,2,5,2003,...,59 rue de l'Abbaye,NaN,Reims,NaN,51100,France,EMEA,Henriot,Paul,Small
2,10134,41,94.74,2,3884.34,7/1/2003 0:00,Shipped,3,7,2003,...,27 rue du Colonel Pierre Avia,NaN,Paris,NaN,75508,France,EMEA,Da Cunha,Daniel,Medium
3,10145,45,83.26,6,3746.70,8/25/2003 0:00,Shipped,3,8,2003,...,78934 Hillside Dr.,NaN,Pasadena,CA,90003,USA,NaN,Young,Julie,Medium
4,10159,49,100.00,14,5205.27,10/10/2003 0:00,Shipped,4,10,2003,...,7734 Strong St.,NaN,San Francisco,CA,NaN,USA,NaN,Brown,Julie,Medium


In [7]:
# df.groupby("QTR_ID").sum()["SALES"]

In [8]:
# what was the total sum of sales per quarter? 
# --> SELECT SUM(SALES) FROM table WHERE...

In [9]:
# pip install sqlalchemy
# sudo pip uninstall sqlalchemy

In [10]:
from sqlalchemy import create_engine
from sqlalchemy import text 

In [11]:
# TEMP DB in RAM
#PUSH Pandas DF --> TEMP DB
# SQL query on TEMP DB

In [12]:
temp_db = create_engine('sqlite:///:memory:', echo=True)

In [13]:
data = df.to_sql(name='Sales', con=temp_db)

2024-02-07 23:54:15,428 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-07 23:54:15,428 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Sales")
2024-02-07 23:54:15,428 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-02-07 23:54:15,429 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Sales")
2024-02-07 23:54:15,429 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-02-07 23:54:15,430 INFO sqlalchemy.engine.Engine ROLLBACK
2024-02-07 23:54:15,431 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-07 23:54:15,432 INFO sqlalchemy.engine.Engine 
CREATE TABLE "Sales" (
	"index" BIGINT, 
	"ORDERNUMBER" BIGINT, 
	"QUANTITYORDERED" BIGINT, 
	"PRICEEACH" FLOAT, 
	"ORDERLINENUMBER" BIGINT, 
	"SALES" FLOAT, 
	"ORDERDATE" TEXT, 
	"STATUS" TEXT, 
	"QTR_ID" BIGINT, 
	"MONTH_ID" BIGINT, 
	"YEAR_ID" BIGINT, 
	"PRODUCTLINE" TEXT, 
	"MSRP" BIGINT, 
	"PRODUCTCODE" TEXT, 
	"CUSTOMERNAME" TEXT, 
	"PHONE" TEXT, 
	"ADDRESSLINE1" TEXT, 
	"ADDRESSLINE2" TEXT, 
	"CITY" TEXT, 
	"STATE" TEXT, 
	

In [14]:
with temp_db.connect() as conn:
    # makes the connection
    # run doce indentation/block of code 
    result = conn.execute(text("SELECT SUM(SALES) FROM Sales"))
    # auto close connection 

2024-02-07 23:54:15,495 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-07 23:54:15,498 INFO sqlalchemy.engine.Engine SELECT SUM(SALES) FROM Sales
2024-02-07 23:54:15,498 INFO sqlalchemy.engine.Engine [generated in 0.00322s] ()
2024-02-07 23:54:15,500 INFO sqlalchemy.engine.Engine ROLLBACK


In [19]:
client = OpenAI()

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [20]:
### sqlite SQL tables, with their properties: 
#
# Employee(id, name, department_id)
# Department(id, name, address)
# Salary_Payments(id, employy_id, amount, data)
#
### A query to list the names of the departments hwich employed more than 10
# SELECT

In [15]:
def create_table_definition(df):
    """
    Create an Introprompt for OpenAI 
    """
    prompt = """### sqlite SQL table , with it properties: 
    #
    # Sakes({})
    #
    """.format(",".join(str(col) for col in df.columns))
    return prompt

In [16]:
",".join(str(col) for col in df.columns)

'ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,MSRP,PRODUCTCODE,CUSTOMERNAME,PHONE,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE'

In [17]:
print(create_table_definition(df))

### sqlite SQL table , with it properties: 
    #
    # Sakes(ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,MSRP,PRODUCTCODE,CUSTOMERNAME,PHONE,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE)
    #
    


In [18]:
# Grab a natural language request 
def prompt_input():

    nlp_text = input("Enter the info you want: ")
    return nlp_text
    

In [19]:
prompt_input()

Enter the info you want: return the sum of SALES per POSTALCODE


'return the sum of SALES per POSTALCODE'

In [20]:
def combine_prompts(df, query_prompt):
    definition = create_table_definition(df)
    query_init_string = f"### A query to answer: {query_prompt} \nSELECT"
    return definition+query_init_string

In [21]:
nlp_text = prompt_input() # NLP
combine_prompts(df, nlp_text) # DF + query that does... + NLP

Enter the info you want: return the sum of SALES per POSTALCODE


'### sqlite SQL table , with it properties: \n    #\n    # Sakes(ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,MSRP,PRODUCTCODE,CUSTOMERNAME,PHONE,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE)\n    #\n    ### A query to answer: return the sum of SALES per POSTALCODE \nSELECT'

In [16]:
# NO LONGER RUNNABLE 

# response = openai.Completion.create(
#             model = 'code-davinci-002',
#             prompt = combine_prompts(df, nlp_text),
#             temperature = 0,
#             max_toekns = 150)

In [ ]:
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": create_table_definition_prompt(df)},
    {"role": "user", "content": f"A query to answer: {nlp_text}"},
  ]
)

In [17]:
response['choices'][0] ['text']

NameError: name 'response' is not defined

In [ ]:
def handle_response(response):
    query = response['chioces'][0]['text']
    if query.startwith(" "):
        query = "SELECT" + query
    return query

In [24]:
handle_response(response)

NameError: name 'handle_response' is not defined

In [25]:
with temp_db.connect() as conn:
    result = conn.execute(text(handle_response(response)))

NameError: name 'handle_response' is not defined

In [ ]:
result.all()